# Neural Creative Transfer

We implement an modification of Gatys, Ecker and Bethge's [neural style transfer algorithm](http://arxiv.org/abs/1508.06576) for our creative assets.

The algorith uses a VGG-19 network to separate and recombine content of an creative and style of a website, and therefore provides a system for creating native creative for advertising. 

In [2]:
import os
import sys

import numpy as np
import scipy.io
import scipy.misc
import tensorflow as tf 

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image

%matplotlib inline

In [33]:
###############################################################################
# Constants
###############################################################################
# Weight to put more emphasis on the content loss.
ALPHA = 5
# Weight to put more emphasis on the style loss.
BETA = 1000
# Max iteration to run the optimizer
MAX_ITERATION = 1000
#Number of iterations between optimizer print statements
PRINT_ITERATION = 50

In [12]:
def setup_network_from_file(path, image):
    """
    We start with the pre-trained VGG model from the paper "Very Deep Convolutional
    Networks for Large-Scale Visual Recognition". We  strip out all the conv, relu
    and pool layers from the VGG-19 network and replace the maxpool layers with 
    average pool layers as suggested by the authors in the "A Neural Algorithm of 
    Artistic Style" paper. Here is the configuration for our modified VGG network 
    (layer : filter height, filter width, input channels, output channels):
        conv1_1  :  3  3  3    64
        conv1_2  :  3  3  64   64 
        avgpool  :
        conv2_1  :  3  3  64   128  
        conv2_2  :  3  3  128  128 
        avgpool  :
        conv3_1  :  3  3  128  256 
        conv3_2  :  3  3  256  256
        conv3_3  :  3  3  256  256
        conv3_4  :  3  3  256  256
        avgpool  :
        conv4_1  :  3  3  256  512  
        conv4_2  :  3  3  512  512
        conv4_3  :  3  3  512  512
        conv4_4  :  3  3  512  512
        avgpool  :
        conv5_1  :  3  3  512  512 
        conv5_2  :  3  3  512  512 
        conv5_3  :  3  3  512  512 
        conv5_4  :  3  3  512  512 
        avgpool  :    
    """
    
    _, image_height, image_width, color_channels = image.shape
    
    # Strip out the layers from the VGG-19 model 
    vgg = scipy.io.loadmat(path)['layers'] 
    
    def _weights(layer):
        """
        Extract the weights at a given layer.
        """
        W = vgg[0][layer][0][0][2][0][0]
        return tf.constant(W)
    
    def _bias(layer):
        """
        Extract the bias at a given layer.
        """
        b = vgg[0][layer][0][0][2][0][1]
        return tf.constant(np.reshape(b, (b.size)))

    def _conv2d(input_layer, layer):
        """
        Construct the conv2d layer.
        """
        W = _weights(layer)
        b = _bias(layer)
        return tf.nn.conv2d(input_layer, filter=W, strides=[1, 1, 1, 1], padding='SAME') + b
    
    def _relu(input_layer):
        """
        Construct the relu layer.
        """
        return tf.nn.relu(input_layer)

    def _avg_pool(input_layer):
        """
        Return the average pooling layer.
        """
        return tf.nn.avg_pool(input_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Set up the modifiled VGG network.  
    network = {}
    network['input']    = tf.Variable(np.zeros((1, image_height, image_width, color_channels)), dtype = np.float32)
    network['conv1_1']  = _relu(_conv2d(network['input'], 0))
    network['conv1_2']  = _relu(_conv2d(network['conv1_1'], 2))    
    network['avgpool1'] = _avgpool(network['conv1_2'])
    network['conv2_1']  = _relu(_conv2d(network['avgpool1'], 5))
    network['conv2_2']  = _relu(_conv2d(network['conv2_1'], 7))
    network['avgpool2'] = _avgpool(network['conv2_2'])
    network['conv3_1']  = _relu(_conv2d(network['avgpool2'], 10))
    network['conv3_2']  = _relu(_conv2d(network['conv3_1'], 12))
    network['conv3_3']  = _relu(_conv2d(network['conv3_2'], 14))
    network['conv3_4']  = _relu(_conv2d(network['conv3_3'], 16))
    network['avgpool3'] = _avgpool(network['conv3_4'])
    network['conv4_1']  = _relu(_conv2d(network['avgpool3'], 19))
    network['conv4_2']  = _relu(_conv2d(network['conv4_1'], 21))
    network['conv4_3']  = _relu(_conv2d(network['conv4_2'], 23))
    network['conv4_4']  = _relu(_conv2d(network['conv4_3'], 25))
    network['avgpool4'] = _avgpool(network['conv4_4'])
    network['conv5_1']  = _relu(_conv2d(network['avgpool4'], 28))
    network['conv5_2']  = _relu(_conv2d(network['conv5_1'], 30))
    network['conv5_3']  = _relu(_conv2d(network['conv5_2'], 32))
    network['conv5_4']  = _relu(_conv2d(network['conv5_3'], 34))
    network['avgpool5'] = _avgpool(network['conv5_4'])
    
    return network

In [36]:
def content_loss_func(sess, network, image, layers = [], weights = []):
    """
    Content loss function.
    """
    def _content_loss(p, x):
        _, height, width, depth = p.get_shape()
        # N is the number of the output channels.
        N = depth.value
        # M is the size of the feature map.
        M = height.value * width.value
        # I use a similar normalization constant to the one used in
        # style loss instead of the one in the paper, which is 0.5
        C = 1. / (2 * N * M)
        return C * tf.reduce_sum(tf.pow(x - p, 2))
    
    sess.run(network['input'].assign(image))
    content_layers = zip(layers, weights)
    content_loss = sum([_content_loss(sess.run(network[layer]), network[layer]) * weight
                        for layer, weight in style_layers])
        
    return content_loss / len(layers)

def style_loss_func(sess, network, image, layers = [], weights = []):
    """
    Style loss function as defined in the paper.
    """
    def _gram_matrix(x, N, M):
        """
        The gram matrix.
        """
        F = tf.reshape(x, (M, N))
        return tf.matmul(tf.transpose(F), F)

    def _style_loss(a, x):
        """
        The style loss calculation.
        """
        _, height, width, depth = a.get_shape()
        # N is the number of the output channels.
        N = depth.value
        # M is the size of the feature map.
        M = height.value * width.value
        # A is the style representation of the original image.
        A = _gram_matrix(a, N, M)
        # G is the style representation of the generated image.
        G = _gram_matrix(x, N, M)
        # normalization factor
        C = 1. / (4 * N**2 * M**2)

        return C * tf.reduce_sum(tf.pow(G - A, 2))

    sess.run(network['input'].assign(image))
    style_layers = zip(layers, weights)
    style_loss = sum([_style_loss(sess.run(network[layer]), network[layer]) * weight
                      for layer, weight in style_layers])
    
    return style_loss

In [37]:
def style_transfer(content_img, style_img, init_img, **kwargs):
  with tf.Session() as sess:
    
    assert content_img.shape == style_imgs.shape
    
    # setup the modifield VGG network
    network = setup_network_from_file(content_img)
    
    # content loss
    L_content = content_loss_func(sess, network, content_img)
    
    # style loss
    L_style = style_loss_func(sess, network, style_img)
    
    # loss weights
    alpha = kwargs.get('alpha', ALPHA)
    beta  = kwargs.get('beta', BETA)
    
    # total loss
    L_total  = alpha * L_content + beta  * L_style
    
    # 'L-BFGS-B' requires much less tuning compared to 'Adam'
    optimizer = tf.contrib.opt.ScipyOptimizerInterface(
        L_total, 
        method='L-BFGS-B',
        options={'maxiter': kwargs.get('maxiter', MAX_ITERATION),
                 'disp': kwargs.get('maxiter', PRINT_ITERATION)})
    
    # Optimizing the loss function
    sess.run(tf.global_variables_initializer())
    sess.run(init_op)
    sess.run(net['input'].assign(init_img))
    optimizer.minimize(sess)
    
    output_img = sess.run(net['input'])
    
    return output_img